<a href="https://colab.research.google.com/github/AkindeKadjo/food_sales_predictions/blob/main/04_Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Akinde Kadjo

Date: 09/15/2022

**We will continue to work on your sales prediction project. The goal of this step is to help the retailer by using machine learning to make predictions about future sales based on the data provided.**

**For Part 5, you will go back to your original, uncleaned, sales prediction dataset with the goal of preventing data leakage.**

In [1]:
#imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')
filename = '/content/drive/MyDrive/Coding Dojo/05 Week 5 Intro to ML/sales_predictions.csv'
Original_df = pd.read_csv(filename)
# keeping a copy of the df
df = Original_df.copy()
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##Data Cleaning

Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data. (There is a way to do this after the split, but for this project, you may perform this step before the split)

In [2]:
#checking for data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [3]:
#check for duplicate and delete them
df.duplicated().sum()
df = df.drop_duplicates()

In [4]:
#check for missing values
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [5]:
#Identify and address any inconsistencies in categorical values
dtypes = df.dtypes
str_cols = dtypes[dtypes=='object'].index
for col in str_cols:
    print(f"- Column= {col}")
    print(df[col].value_counts(dropna=False))
    print('\n\n')

- Column= Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



- Column= Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Column= Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



- Column= Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    9

In [6]:
#fix the inconcistency when it comes to Item fat content
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('reg','Regular')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('low fat', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('LF', 'Low Fat')
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [7]:
#checking for significant outliers for the numerical data
df.describe().round(2)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.00,8523.00,8523.00,8523.00,8523.00
mean,12.86,0.07,140.99,1997.83,2181.29
std,4.64,0.05,62.28,8.37,1706.50
min,4.56,0.00,31.29,1985.00,33.29
25%,8.77,0.03,93.83,1987.00,834.25
50%,12.60,0.05,143.01,1999.00,1794.33
75%,16.85,0.09,185.64,2004.00,3101.30
max,21.35,0.33,266.89,2009.00,13086.96


##Data Splitting and Preprocessing

Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.  

In [8]:
#set y as a target column
y = df['Item_Outlet_Sales']
#set X as the feature data, decision made based on prior analysis results from previous plots
X = df[['Item_Visibility','Item_Type','Item_MRP', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]
X.head()

,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,0.016047,Dairy,249.8092,Medium,Tier 1,Supermarket Type1
1,0.019278,Soft Drinks,48.2692,Medium,Tier 3,Supermarket Type2
2,0.016760,Meat,141.6180,Medium,Tier 1,Supermarket Type1
3,0.000000,Fruits and Vegetables,182.0950,NaN,Tier 3,Grocery Store
4,0.000000,Household,53.8614,High,Tier 3,Supermarket Type1


Perform a train test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

Creating a preprocessing object to prepare the dataset for Machine Learning

In [10]:
#quick check of the data type
X.dtypes

Item_Visibility         float64
Item_Type                object
Item_MRP                float64
Outlet_Size              object
Outlet_Location_Type     object
Outlet_Type              object
dtype: object

>Numerical columns are: Item_Visibility, Item_MRP

>Nominal columns are: Item_Type, Outlet_Type

>Ordinal columns are: Outlet_Size, Outlet_Location_Type

Making sure my imputation of missing values occurs  after the train test split using SimpleImputer. 

In [11]:
#instantiate the StandardScaler, OneHotEncoder and Imputers
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

# Instantiating the OrdinalEncoder 
  # Creating a list of ordinal labels in acsending order
Size_labels = ['Small','Medium','High']
Location_labels = ['Tier 3','Tier 2','Tier 1']
  #combining the ordered list in the order that the columns appear
ordered_labels = [Size_labels, Location_labels]
ordinal = OrdinalEncoder(categories = ordered_labels)

Creating a Pipeline Numeric Data and a Pipeline for Categorical Data

In [13]:
# Setup the pipelines for the numeric and categorical data
num_pipeline = make_pipeline(mean_imputer, scaler)
ord_pipeline = make_pipeline(freq_imputer, ordinal)
nom_pipeline = make_pipeline(freq_imputer, ohe)

Creating Tuples to Pair Pipelines with Columns

In [14]:
# Create column lists for objects and a number selector
ordinal_cols = ['Outlet_Size', 'Outlet_Location_Type']
nominal_cols = ['Item_Type', 'Outlet_Type']
num_selector = make_column_selector(dtype_include='number')

# Setup the tuples to pair the processors with the make column selectors
numeric_tuple = (num_pipeline, num_selector)
ordinal_tuple = (ord_pipeline, ordinal_cols)
nominal_tuple = (nom_pipeline, nominal_cols)

Creating Column Transformer and fitting it on the Training Data

In [15]:
# Instantiate the make column transformer
preprocessor = make_column_transformer(ordinal_tuple, numeric_tuple, nominal_tuple, remainder='drop')
# Fit the column transformer on the X_train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Tier '
                                                                              '3',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '1']]))]),
                                 ['Outlet_Size', 'Outlet_Location_Type']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa3fcd82310>),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Item_Type', 'Outlet_Type'])])

Transforming Both Training and Testing Data

In [16]:
# Transform the X_train and the X_test
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

Checking the result

In [17]:
X_train_transformed.shape

(6392, 24)

In [18]:
X_train_transformed

array([[ 1.        ,  0.        , -0.71277507, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.        , -1.29105225, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  2.        ,  1.81331864, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        ,  1.        , -0.92052713, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        , -0.2277552 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.        , -0.95867683, ...,  1.        ,
         0.        ,  0.        ]])

In [19]:
#showing the data in pandas dataframe 
X_train_transformed_df = pd.DataFrame(X_train_transformed)
X_train_transformed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,-0.712775,1.828109,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,-1.291052,0.603369,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,2.0,1.813319,0.244541,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,-1.004931,-0.952591,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1.0,-0.965484,-0.336460,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
